In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from xgboost import XGBClassifier

## Reading training data

In [2]:
df = pd.read_csv("train.csv")
df.head()

,id,created_at,campaign_var_1,campaign_var_2,products_purchased,signup_date,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy
0,1,2021-01-01,1,2,2.0,2020-09-24,0,0,0,0,0,0,1,1,0,0,0,0,0
1,2,2021-01-01,2,1,2.0,2020-09-19,1,0,1,0,0,0,1,0,0,0,0,0,0
2,3,2021-01-01,9,3,3.0,2021-08-11,1,0,0,0,0,0,0,0,0,0,0,0,0
3,4,2021-01-01,6,7,2.0,2017-10-04,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,2021-01-01,4,6,NaN,2020-06-08,0,0,0,0,0,0,1,0,0,0,1,0,0


## Cleaning Data 

In [3]:
df['products_purchased'] = df['products_purchased'].fillna(0.0)

In [4]:
df = df.drop(['signup_date'], axis=1)

## Generating timeseries related columns

In [5]:
df['day'] = pd.to_datetime(df['created_at']).dt.day
df['month'] = pd.to_datetime(df['created_at']).dt.month
df['day_name'] = pd.to_datetime(df['created_at']).dt.day_name()

In [6]:
## Prepared dataFrame
df.head()

,id,created_at,campaign_var_1,campaign_var_2,products_purchased,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,...,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy,day,month,day_name
0,1,2021-01-01,1,2,2.0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,1,Friday
1,2,2021-01-01,2,1,2.0,1,0,1,0,0,...,1,0,0,0,0,0,0,1,1,Friday
2,3,2021-01-01,9,3,3.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,Friday
3,4,2021-01-01,6,7,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,Friday
4,5,2021-01-01,4,6,0.0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,1,Friday


In [7]:
## Removing unwanted column now
df = df.drop(['created_at'], axis=1)

In [8]:
# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
df['day_name']= label_encoder.fit_transform(df['day_name'])

In [9]:
df.head()

,id,campaign_var_1,campaign_var_2,products_purchased,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy,day,month,day_name
0,1,1,2,2.0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0
1,2,2,1,2.0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0
2,3,9,3,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,4,6,7,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,5,4,6,0.0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0


## Preparing source and target columns

In [10]:
x = df.drop(['id', 'buy'], axis=1)
y = df['buy']

## Building model as per the output which we got after grid CV

In [11]:
xgBoost_model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1,
              objective='binary:hinge', predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## Fitting the model

In [12]:
xgBoost_model.fit(x,y)

C:\Users\Guru Saluja\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:34:06] WARNING: ..\src\learner.cc:576: 
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1,
              objective='binary:hinge', predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## Reading test data

In [13]:
test_df = pd.read_csv("test.csv")

## Cleaning testing data

In [14]:
test_df['products_purchased'] = test_df['products_purchased'].fillna(0.0)
test_df = test_df.drop(['signup_date'], axis=1)

test_df['day'] = pd.to_datetime(test_df['created_at']).dt.day
test_df['month'] = pd.to_datetime(test_df['created_at']).dt.month
test_df['day_name'] = pd.to_datetime(test_df['created_at']).dt.day_name()

## Removing unwanted column now
test_df = test_df.drop(['created_at'], axis=1)

# Encode labels in column 'day_name'.
test_df['day_name']= label_encoder.transform(test_df['day_name'])

test_df.head()

,id,campaign_var_1,campaign_var_2,products_purchased,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,day,month,day_name
0,39162,2,2,0.0,1,1,0,0,0,1,0,0,0,0,1,0,1,1,2
1,39163,4,7,3.0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,2
2,39164,8,7,0.0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,2
3,39165,9,8,2.0,0,0,0,0,1,1,1,0,0,0,2,0,1,1,2
4,39166,4,5,2.0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,2


In [15]:
x_test_df = test_df.drop(['id'], axis=1)

## applying prediction function model 

In [16]:
y_pred = xgBoost_model.predict(x_test_df)

In [17]:
submitted_out = {
    "id": test_df['id'],
    "buy": y_pred
}

out_df = pd.DataFrame(submitted_out)
out_df.head()

,id,buy
0,39162,1
1,39163,0
2,39164,0
3,39165,1
4,39166,0


## Saving the result

In [18]:
out_df.to_csv("Result.csv", index=False)

## Hyper parameter tunning Block

In [19]:
# This method will do Grid search cross validation (cross validate technique is Shuffle split and split size is 5)
# on model to find best hyperpermeter as per the which it gets

def getHyperparmeter(model, hypercombination, scoring):
    ss = ShuffleSplit(n_splits=5, test_size=0.1, random_state=0)
    return GridSearchCV(model, hypercombination, scoring=scoring, cv=ss)

In [20]:
## Possible combination for parameter

XGBoost_hyper = {
 'booster': ['gbtree', 'dart'],
 'learning_rate': [0.01, 0.03, 0.1, 0.3],
 'max_depth': [6, 10],
 'colsample_bytree':[0.03, 0.5, 1],
 'objective': ['binary:logitraw', 'binary:logistic', 'binary:hinge'],
 'eval_metric': ['auc', None],
 'n_estimators': [1000, 2000]
 }

In [21]:
xgBoost_model = XGBClassifier()
xgBoost_CV = getHyperparmeter(xgBoost_model, XGBoost_hyper, 'f1')
xgBoost_CV.fit(x, y)
xgBoost_CV.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)